In [31]:
import pandas as pd
from scipy.stats import shapiro, levene, f_oneway, ttest_ind, binomtest

df = pd.read_csv("Titanic-Dataset.csv")


In [32]:
women_survived = df[(df['Sex'] == 'female') & (df['Survived'] == 1)]['Age'].dropna()
women_not_survived = df[(df['Sex'] == 'female') & (df['Survived'] == 0)]['Age'].dropna()
men_survived = df[(df['Sex'] == 'male') & (df['Survived'] == 1)]['Age'].dropna()
men_not_survived = df[(df['Sex'] == 'male') & (df['Survived'] == 0)]['Age'].dropna()

print("\nПроверка нормальности распределения (тест Шапиро-Уилка):")


Проверка нормальности распределения (тест Шапиро-Уилка):


In [33]:
shapiro_women_survved = shapiro(women_survived)
print(f"Выжившие женщины: статистика={shapiro_women_survived.statistic:.4f}, p-value={shapiro_women_survived.pvalue:.4f}")


Выжившие женщины: статистика=0.9828, p-value=0.0163


In [34]:
shapiro_women_not_survived = shapiro(women_not_survived)
print(f"Погибшие женщины: статистика={shapiro_women_not_survived.statistic:.4f}, p-value={shapiro_women_not_survived.pvalue:.4f}")

Погибшие женщины: статистика=0.9739, p-value=0.1928


In [35]:
shapiro_men_survived = shapiro(men_survived)
print(f"Выжившие мужчины: статистика={shapiro_men_survived.statistic:.4f}, p-value={shapiro_men_survived.pvalue:.4f}")

Выжившие мужчины: статистика=0.9621, p-value=0.0085


In [36]:
shapiro_men_not_survived = shapiro(men_not_survived)
print(f"Погибшие мужчины: статистика={shapiro_men_not_survived.statistic:.4f}, p-value={shapiro_men_not_survived.pvalue:.4f}")


Погибшие мужчины: статистика=0.9593, p-value=0.0000


In [37]:
print("\nПроверка однородности дисперсий (тест Левина):")


Проверка однородности дисперсий (тест Левина):


In [38]:
levene_result = levene(women_survived, women_not_survived, men_survived, men_not_survived)
print(f"Статистика={levene_result.statistic:.4f}, p-value={levene_result.pvalue:.4f}")

Статистика=1.0647, p-value=0.3634


In [39]:
alpha = 0.05
all_normal = all(p > alpha for p in [
    shapiro_women_survived.pvalue,
    shapiro_women_not_survived.pvalue,
    shapiro_men_survived.pvalue,
    shapiro_men_not_survived.pvalue
])


In [40]:
if all_normal:
    print("\nВсе распределения можно считать нормальными (p > 0.05)")
    print("Применяем ANOVA согласно требованиям задачи:")
    anova_result = f_oneway(women_survived, women_not_survived, men_survived, men_not_survived)
    print(f"Статистика F={anova_result.statistic:.4f}, p-value={anova_result.pvalue:.4f}")

    if anova_result.pvalue < alpha:
        print(f"Вывод: p-value ({anova_result.pvalue:.4f}) < {alpha}, поэтому отвергаем нулевую гипотезу.")
        print("Возрастные распределения в группах статистически различны.")
    else:
        print(f"Вывод: p-value ({anova_result.pvalue:.4f}) > {alpha}, поэтому принимаем нулевую гипотезу.")
        print("Возрастные распределения в группах статистически одинаковы.")
else:
    print("\nНе все распределения можно считать нормальными (p < 0.05)")
    print("Применяем биномиальный тест согласно требованиям задачи.")


Не все распределения можно считать нормальными (p < 0.05)
Применяем биномиальный тест согласно требованиям задачи.


In [41]:
median_age = pd.concat([women_survived, women_not_survived, men_survived, men_not_survived]).median()
print(f"Медианный возраст всех пассажиров: {median_age:.2f}")

ws_binary = (women_survived > median_age).sum()
wns_binary = (women_not_survived > median_age).sum()
ms_binary = (men_survived > median_age).sum()
mns_binary = (men_not_survived > median_age).sum()

print("\nРезультаты биномиальных тестов (проверка пропорции пассажиров старше медианного возраста):")

binom_ws = binomtest(ws_binary, len(women_survived), 0.5)
binom_wns = binomtest(wns_binary, len(women_not_survived), 0.5)
binom_ms = binomtest(ms_binary, len(men_survived), 0.5)
binom_mns = binomtest(mns_binary, len(men_not_survived), 0.5)

print(f"Выжившие женщины: доля старше медианы={ws_binary/len(women_survived):.4f}, p-value={binom_ws.pvalue:.4f}")
print(f"Погибшие женщины: доля старше медианы={wns_binary/len(women_not_survived):.4f}, p-value={binom_wns.pvalue:.4f}")
print(f"Выжившие мужчины: доля старше медианы={ms_binary/len(men_survived):.4f}, p-value={binom_ms.pvalue:.4f}")
print(f"Погибшие мужчины: доля старше медианы={mns_binary/len(men_not_survived):.4f}, p-value={binom_mns.pvalue:.4f}")


print("\nВывод на основе биномиальных тестов:")
significant_diff = any(p < alpha for p in [binom_ws.pvalue, binom_wns.pvalue, binom_ms.pvalue, binom_mns.pvalue])
if significant_diff:
    print("Обнаружены статистически значимые различия в возрастных распределениях групп.")
    print("По крайней мере в одной группе пропорция пассажиров старше медианного возраста")
    print("значимо отличается от ожидаемой при равномерном распределении.")
else:
    print("Не обнаружено статистически значимых различий в возрастных распределениях групп.")
    print("Во всех группах пропорция пассажиров старше медианного возраста")
    print("статистически не отличается от ожидаемой при равномерном распределении.")



Медианный возраст всех пассажиров: 28.00

Результаты биномиальных тестов (проверка пропорции пассажиров старше медианного возраста):
Выжившие женщины: доля старше медианы=0.4924, p-value=0.8867
Погибшие женщины: доля старше медианы=0.3750, p-value=0.0599
Выжившие мужчины: доля старше медианы=0.4839, p-value=0.8358
Погибшие мужчины: доля старше медианы=0.5167, p-value=0.5621

Вывод на основе биномиальных тестов:
Не обнаружено статистически значимых различий в возрастных распределениях групп.
Во всех группах пропорция пассажиров старше медианного возраста
статистически не отличается от ожидаемой при равномерном распределении.
